In [7]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np

In [60]:
reg_neurons_path =  '/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Outputs_200/7_Gallery_Neurons/'
features_path = '/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Outputs_200/8_Features/'
code_file = '/nas1/Data_Mythreya/MotorCortexProject/V8/code/NeuroFeature/hocreader.py'

In [61]:
os.system('source /home/BuildingBrains/canopy/bin/activate')


0

In [62]:
for file in glob.glob('/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/*'):
    print(file)
    os.system('python {} {} {}'.format(code_file,file,features_path))

/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130319_2_NeuroMap_ready.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130320_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130402.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130404_1.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130404_2_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130507_1_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130507_2_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20130508_1_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_

/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_1_Cell_B.am_Segmented_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_1_Cell_C_done.am_Segmented_corrected.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_2_Cluster_A_Cell_A_done.am_Segmented_corrected_NeuroMap_ready.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_2_Cluster_A_Cell_B_done.am_Segmented_corrected_NeuroMap_ready.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_2_Cluster_A_Cell_C_done.am_Segmented_corrected_NeuroMap_ready.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Input_Spatial_Graphs/All_Hocs/RA20180109_2_Cluster_B_Cell_A_done.am_Segmented_corrected_NeuroMap_ready.hoc
/nas1/Data_Mythreya/MotorCortexProject/V8/Regis

In [20]:
# Feature extraction
features_cols = [\
    "Experiment_ID"\
    ,"Soma_Depth_From_Pia"
    , "Dendritic_Length"  \
    , "Dendritic_Volume"  \
    , "Dendritic_Width"  \
    , "Dendritic_Height"  \
    , "Dendritic_Branchpoints"  \
    , "Dendritic_Branchorder_Avg"  \
    , "Dendritic_Branchorder_Max"  \
    , "Dendritic_Endpoint_Avg"  \
    , "Dendritic_Endpoint_Max"  \
    , "Dendritic_Endpoint_Sum"  \
    , "Dendritic_Endpoint_Centroid"  \
    , "Apical_Length"  \
    , "Apical_Volume"  \
    , "Apical_Width"  \
    , "Apical_Height"  \
    , "Apical_Branchpoints"  \
    , "Apical_Endpoint_Centroid"  \
    , "Basal_Length"  \
    , "Basal_Volume"  \
    , "Basal_Width"  \
    , "Basal_Height"  \
    , "Basal_Branchpoints"  \
    , "Axon_Branchpoints"
    , "Axon_Length"
    , "Axon_Volume" ]

In [ ]:
df_features = []


In [11]:
filename = sorted(glob.glob(reg_neurons_path+'/*'))[0]
sg = AmiraSpatialGraph(filename)

In [25]:
filename

'/nas1/Data_Mythreya/MotorCortexProject/V8/Registration_Local/Outputs_200/7_Gallery_Neurons/RA20130319_2.am'

In [18]:
sg.neuron.dendrite.basal_dendrite.edge_list[0]

[[-6.856946382228841, -5.530016015730425, -608.8723952153409],
 [-6.693106204727769, -6.695400431171535, -609.0669088664915],
 [-6.4648321189138755, -7.848117998133347, -608.9077449492141],
 [-7.076248289719956, -8.65436726856018, -609.5831947104933]]

In [24]:
soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
soma_depth = abs(soma_center[2])

In [30]:
len(sg.neuron.dendrite.basal_dendrite.vertices)

76

In [49]:
def get_root_branching_terminal_nodes(graph_data,soma_center):
    root_nodes = []
    end_nodes = []
    branch_nodes = []
    for i in range(len(graph_data.vertices)):
        vert = graph_data.vertices[i]
        # find vertices that have only one connectivity
        # see how many times this vertex appears in the from vert position of connectivity list
        num_edge_eminating_from = 0
        num_edge_ending_at = 0
        for connectivity in graph_data.edge_connectivity:
            if i == connectivity[0]:
                num_edge_eminating_from = num_edge_eminating_from + 1
            if i == connectivity[1]:
                num_edge_ending_at = num_edge_ending_at + 1
        if num_edge_eminating_from == 1 and distance.euclidean(soma_center,vert) < 25:
            root_nodes.append(vert)
        if num_edge_eminating_from > 1:
            branch_nodes.append(vert)
        if num_edge_eminating_from == 0:
            end_nodes.append(vert)
    print('# Root Nodes = {}'.format(len(root_nodes)))
    print('# Branch Nodes = {}'.format(len(branch_nodes)))
    print('# Terminal Nodes = {}'.format(len(end_nodes)))
        
    return root_nodes,branch_nodes,end_nodes

In [55]:
root_nodes,branch_nodes,end_nodes = get_root_branching_terminal_nodes(sg.neuron.dendrite.basal_dendrite,soma_center)

# Root Nodes = 4
# Branch Nodes = 34
# Terminal Nodes = 38


In [ ]:
graph_data = sg.neuron.dendrite.basal_dendrite
tree_depths = []
for i in  range(len(root_nodes)):
    root_vert = root_nodes[i]
    tree_depths.append(0)
    for connectivity in graph_data.edge_connectivity:
        if root_vert == graph_data.vertices[connectivity[0]]:
            # this is the starting edge
            
        